<a href="https://colab.research.google.com/github/mistryvivek/YRKCS-PRBX/blob/main/Model_1_Basic_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import random
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import OneHotEncoder

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


# Setup Weights and Bias.

* AI experimental tool.

In [ ]:
!pip install wandb -qU

In [ ]:
# Log in to your W&B account
import wandb
import random
import math

In [ ]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Make RNN

In [ ]:
class RNN(nn.Module):
    def __init__(self,max_laps,hidden_size):
        super(RNN, self).__init__()
        self.embedding =  nn.Embedding(max_laps, hidden_size)
        self.linear_h = nn.Linear(hidden_size,hidden_size)
        self.linear_y = nn.Linear(hidden_size,max_laps)
        self.tahn = nn.Tanh()

    def forward(self,x,hprev):
        h = self.tahn(self.embedding(x) + self.linear_h(hprev))
        y = self.linear_y(h)
        return h,y

# Loss function (Based upon wk11 practical)

* Vocab size is not relevent here - going to give it binary classification values anyway.
* Everything else can be applied for time series: use Cross Entropy Loss, Length of the race = seq_len.
* For each lap, pass through the RNN with previous hidden state and build up hte matrix of outputs.
* Detact the previous hidden state as a new one will be generated.

In [ ]:
def calculate_loss(model,inputs,targets,hprev,max_tyre_life):
  loss_func = nn.BCEWithLogitsLoss()
  seq_length = len(inputs)
  #outputs = torch.zeros(0, max_laps)
  #print(outputs.shape)
  for t in range(seq_length):
    # For each character in the input sequence, pass through RNN with previous hidden state
    hprev,y = model(inputs[t], hprev)
    # Gradually build up matrix of output logits of size seq_length * max_tyre_life
    #outputs = torch.cat((outputs ,y))

  y = y[-1]

  # Compute cross entropy loss for seq_length actual targets against estimated distributions
  loss = loss_func(y,targets.squeeze(0))

  # For truncated backprop, the next subsequence will use the final hidden state
  # but will not backprop through it so we need to detach
  hprev = hprev.detach()

  return loss, hprev

# My dataset - how to format?

* Training and testing
* inputs_training/ inputs_testing/  outputs_testing/ output_training
* For now, it will all just be a 2d array.

ERROR:

```2551     return torch.embedding(weight, input, padding_idx, scale_grad_by_freq, sparse)
   2552
   2553

IndexError: index out of range in self```

In [ ]:
BASE_PATH = r"/content/drive/MyDrive/prbx_data/v1/"
max_race_size = 0

# Define the logical ordering of compounds
compound_order = {'SOFT': 1, 'MEDIUM': 2, 'HARD': 3, 'INTERMEDIATE': 4, 'WET': 5}

In [ ]:
# TRAINING
training_inputs = []
training_outputs = []

RaceCalender22 = pd.read_csv(BASE_PATH + r"2022/eventCalender2022.csv")
for _, row in RaceCalender22.iterrows():
    if row['EventFormat'] != 'testing':
        # Load race data
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2022/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            # Filter data for the specific driver and sort by LapNumber
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            # Extract tyre life and stint columns
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values
            compound_array = TempRaceLoadDriver['Compound'].values

            # Map compounds to ordinal values
            compound_encoded = np.array([compound_order[compound] for compound in compound_array])

            # Create the stint change array
            stint_change_array = []
            for i in range(len(stint_array)):
                if i == len(stint_array) - 1:  # Last lap
                    stint_change_array.append(False)  # Treat as no change
                else:
                    stint_change_array.append(stint_array[i] != stint_array[i + 1])

            stint_change_array = np.array(stint_change_array)

            # Ensure tyre_life_array and stint_change_array are properly formatted
            tyre_life_array = torch.tensor(tyre_life_array, dtype=torch.long, device=device)
            compound_encoded = torch.tensor(compound_encoded, dtype=torch.long, device=device).unsqueeze(1)
            stint_change_array = torch.tensor(stint_change_array, dtype=torch.float32, device=device)

            combined_inputs = torch.cat([tyre_life_array.unsqueeze(1), compound_encoded], dim=1)

            # Add to training data
            training_inputs.append(combined_inputs)
            training_outputs.append(stint_change_array)

In [ ]:
# TESTING
testing_inputs = []
testing_outputs = []

RaceCalender23 = pd.read_csv(BASE_PATH + r"2023/eventCalender2023.csv")
for _, row in RaceCalender23.iterrows():
    if row['EventFormat'] != 'testing':
        # Load race data
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2023/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            # Filter data for the specific driver and sort by LapNumber
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            # Extract tyre life and stint columns
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values
            compound_array = TempRaceLoadDriver['Compound'].values

            # Map compounds to ordinal values
            compound_encoded = np.array([compound_order[compound] for compound in compound_array])

            # Create the stint change array
            stint_change_array = []
            for i in range(len(stint_array)):
                if i == len(stint_array) - 1:  # Last lap
                    stint_change_array.append(False)  # Treat as no change
                else:
                    stint_change_array.append(stint_array[i] != stint_array[i + 1])

            stint_change_array = np.array(stint_change_array)

            # Ensure tyre_life_array and stint_change_array are properly formatted
            tyre_life_array = torch.tensor(tyre_life_array, dtype=torch.long, device=device)
            compound_encoded = torch.tensor(compound_encoded, dtype=torch.long, device=device).unsqueeze(1)
            stint_change_array = torch.tensor(stint_change_array, dtype=torch.float32, device=device)

            combined_inputs = torch.cat([tyre_life_array.unsqueeze(1), compound_encoded], dim=1)

            # Add to testing data
            testing_inputs.append(combined_inputs)
            testing_outputs.append(stint_change_array)

# Training the model

* https://developers.google.com/machine-learning/crash-course/overfitting/interpreting-loss-curves

In [ ]:
hidden_size = 10
lr = 0.01
iterations = 10000
max_race_size = int(max_race_size)
input_parameters = ['TyreLife']
dataset = 'v1'

lr_experiments = [0.01, 0.05, 0.001, 0.005]

In [ ]:
def evaluate_model(inputs, outputs, tolerance=3):
    total_correct = 0
    total_predictions = 0

    for seq_idx in range(len(inputs)):
        # Initialize hidden state for each sequence
        hprev = torch.zeros(hidden_size, device=device)

        # Get the input and target sequences
        input = inputs[seq_idx]
        output = outputs[seq_idx].cpu().numpy()

        # Step through the input sequence and get predicted probabilities
        for t in range(len(input)):
            hprev, y = model(torch.tensor(input[t], device=device), hprev)
            predicted_pits = torch.sigmoid(y) # Apply sigmoid to logits to get probabilities

        y = y[-1]

        predicted_pits = predicted_pits.cpu().detach().numpy()[0]
        predicted_pits = predicted_pits > 0.5
        actual_pits = output == 1.0

        # Now check accuracy for this sequence with tolerance
        correct_count = 0
        total_predictions_for_sequence = 0

        for predicted_pit_idx in range(len(predicted_pits)):
          if predicted_pits[predicted_pit_idx]:  # If a pit stop is predicted
              total_predictions += 1

              # Calculate the tolerance window around the predicted pit
              start_idx = max(0, predicted_pit_idx - tolerance)
              end_idx = min(len(actual_pits), predicted_pit_idx + tolerance + 1)

              # Check if there's any actual pit stop within this window using NumPy
              if np.any(actual_pits[start_idx:end_idx] == 1):  # If there's a pit stop in this window
                  correct_count += 1
                  break

        total_correct += correct_count

    # Calculate overall accuracy
    accuracy = total_correct / total_predictions if total_predictions > 0 else 0
    return accuracy * 100

In [ ]:
def pad_with_last_value(tensor, max_race_size):
    if len(tensor) >= max_race_size:
        return tensor

    last_value = tensor[-1]

    # TO FIX: RuntimeError: a Tensor with 6 elements cannot be converted to Scalar
    # After adding compound
    if last_value.ndimension() > 0:
        padding = last_value.unsqueeze(0).repeat(max_race_size - len(tensor), 1)
    else:
        padding = torch.full((max_race_size - len(tensor),), last_value.item(), dtype=tensor.dtype, device=device)

    return torch.cat((tensor, padding))

In [ ]:
optimizers = {
    "SGD": torch.optim.SGD,  # Stochastic Gradient Descent
    #"RMSprop": torch.optim.RMSprop,  # Root Mean Square Propagation
    #"Adagrad": torch.optim.Adagrad,  # Adaptive Gradient Algorithm
    #"Adadelta": torch.optim.Adadelta,  # Adaptive Delta
    #"AdamW": torch.optim.AdamW,  # Adam with decoupled weight decay
    #"NAdam": torch.optim.NAdam,  # Nesterov-accelerated Adaptive Moment Estimation
    #"ASGD": torch.optim.ASGD,  # Averaged Stochastic Gradient Descent
    #"SparseAdam": torch.optim.SparseAdam,  # Optimizer for sparse tensors
    #"Rprop": torch.optim.Rprop,  # Resilient Backpropagation
}

permutation = torch.randperm(len(training_inputs))
training_inputs = [training_inputs[i] for i in permutation]
training_outputs = [training_outputs[i] for i in permutation]

for lr in lr_experiments:
  model = RNN(max_race_size, hidden_size)
  model = model.to(device)

  optim = optimizers["SGD"](model.parameters(), lr = lr)


  # Initialize WandB
  wandb.init(
      project="prbx-model-v1",
      config={
          "learning_rate": lr,
          "iterations": iterations,
          "hidden_size": hidden_size,
          "max_race_size": max_race_size,
          "input_parameters": input_parameters,
          "dataset": dataset,
          "optimizer": "SGD",
          "random_perms": True,
          "compound_formatting": "logical_ordering",
          "tanh_alternative": "relu",
          "gradient_clipping_method": "fixed 1.0 max norm"
      }
  )

  n, p = 0, 0
  while n<=iterations:
    hprev = torch.zeros(hidden_size,) # reset RNN memory

    inputs = training_inputs[p]
    targets = training_outputs[p]

    inputs = pad_with_last_value(inputs, max_race_size).to(device)
    targets = pad_with_last_value(targets, max_race_size).to(device)
    hprev = torch.zeros(hidden_size, device=device)

    # Compute loss for current subsequence
    loss, hprev = calculate_loss(model,inputs,targets,hprev,max_race_size)
    optim.zero_grad()
    loss.backward()

    # Clip gradients
    max_norm = 1.0  # Maximum norm for gradients
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

    optim.step()

    if n % 500 == 0:
      print('[{:}] Loss: {:.2f}'.format(n,loss.item()))
      testing_accuracy = evaluate_model(testing_inputs,testing_outputs)
      training_accuracy = evaluate_model(training_inputs,training_outputs)

      # Log metrics to WandB
      wandb.log({
          "iteration": n,
          "loss": loss.item(),
          "training_accuracy": training_accuracy,
          "testing_accuracy": testing_accuracy,
      })

      # Save to weights and biases.
      model_path = "model.pth"
      torch.save(model.state_dict(), model_path)


      artifact = wandb.Artifact("basic-rnn-binary-classifier", type="model")
      artifact.add_file(model_path)
      wandb.log_artifact(artifact)

    n += 1 # iteration counter
    p += 1 # dataset counter

    #Restart to the front of the list.
    if len(training_inputs) - 1 == p:
      p = 0
      permutation = torch.randperm(len(training_inputs))
      training_inputs = [training_inputs[i] for i in permutation]
      training_outputs = [training_outputs[i] for i in permutation]

#How can we calculate accuracy for this basic model?

* Say is probability is above 80% percent pit, otherwise don't pit.
* Want at least one right pit stop identification to class it as match.

In [ ]:
for t in range(len(testing_inputs[1])):
  # For each character in the input sequence, pass through RNN with previous hidden state
  hprev,y = model(torch.tensor(testing_inputs[1][t], device=device),hprev)
probabilities = torch.sigmoid(y)

In [ ]:
probabilities[-1]

In [ ]:
for t in range(len(training_inputs[0])):
  # For each character in the input sequence, pass through RNN with previous hidden state
  hprev,y = model(torch.tensor(training_inputs[0][t], device=device),hprev)
probabilities = torch.sigmoid(y)

In [ ]:
probabilities[-1]

# Plot to see what this basic model has remembered - probably just averages

# Validating my data

In [ ]:
len(training_inputs) == len(training_outputs)

In [ ]:
training_inputs[0].shape

In [ ]:
training_outputs[0].shape

## Modify code for sliding window approach (like wk11 code).

* Look at the full race image instead.
* What data types are in the wk11 practical.
* No vocab size required because they apply to catogorical contexts.

In [ ]:
BASE_PATH = r"/content/drive/MyDrive/prbx_data/v1/"
max_race_size = 0
max_tyre_life = 0

In [94]:
# TRAINING
training_inputs = []
training_outputs = []

RaceCalender22 = pd.read_csv(BASE_PATH + r"2022/eventCalender2022.csv")
for _, row in RaceCalender22.iterrows():
    if row['EventFormat'] != 'testing':
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2022/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            if max(TempRaceLoadDriver['TyreLife']) > max_tyre_life:
                max_tyre_life = max(TempRaceLoadDriver['TyreLife'])

            stint_change_array = [0 if stint_array[i] == stint_array[i + 1]else 1
                                  for i in range(len(stint_array) - 1)] + [0]
            stint_change_array = np.array(stint_change_array)

            training_inputs.append(tyre_life_array)
            training_outputs.append(stint_change_array)

In [95]:
# TESTING
testing_inputs = []
testing_outputs = []

RaceCalender23 = pd.read_csv(BASE_PATH + r"2023/eventCalender2023.csv")
for _, row in RaceCalender23.iterrows():
    if row['EventFormat'] != 'testing':
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2023/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            if max(TempRaceLoadDriver['TyreLife']) > max_tyre_life:
                max_tyre_life = max(TempRaceLoadDriver['TyreLife'])

            stint_change_array = [0 if stint_array[i] == stint_array[i + 1]else 1
                                  for i in range(len(stint_array) - 1)] + [0]
            stint_change_array = np.array(stint_change_array)

            testing_inputs.append(tyre_life_array)
            testing_outputs.append(stint_change_array)

In [96]:
class RNNv1b(nn.Module):
    def __init__(self,vocab_size,hidden_size):
        super(RNNv1b, self).__init__()
        self.embedding =  nn.Embedding(vocab_size, hidden_size)
        self.linear_h = nn.Linear(hidden_size,hidden_size)
        self.linear_y = nn.Linear(hidden_size,1)
        self.tanh = nn.Tanh()

    def forward(self,x,hprev):
        h = self.tanh(self.embedding(x) + self.linear_h(hprev))
        y = self.linear_y(h)
        return h,y

In [97]:
def calculate_loss(model,inputs,targets,hprev,vocab_size):
  loss_func = nn.BCEWithLogitsLoss()
  seq_length = len(inputs)
  outputs = []
  for t in range(seq_length):
    # For each character in the input sequence, pass through RNN with previous hidden state
    hprev,y = model(torch.tensor([inputs[t]], device=device),hprev)
    # Gradually build up matrix of output logits of size seq_length * vocab_size - we want it at every time step do not want this.
    # Compute cross entropy loss for seq_length actual targets against estimated distributions
    # RuntimeError: result type Float can't be cast to the desired output type Long
    outputs.append(y)

  outputs = torch.stack(outputs).squeeze(1).squeeze(1)
  loss = loss_func(outputs ,targets)

  # For truncated backprop, the next subsequence will use the final hidden state
  # but will not backprop through it so we need to detach
  hprev = hprev.detach()

  return loss, hprev

In [98]:
# Model parameters to be logged by w and b.
hidden_size = 10
lr = 0.001
iterations = 5000
input_parameters = ['TyreLife']
dataset = 'v1'
vocab_size = int(max_tyre_life) + 1 # Possiblity for each tyre_life

In [99]:
# PADDING
def add_padding(max_race_size, sequence):
    if len(sequence) >= max_race_size:
        return sequence[:max_race_size]
    else:
        padding = [0] * (max_race_size - len(sequence))
        return np.concatenate([sequence,padding])

In [100]:
model = RNNv1b(vocab_size,hidden_size)
model.to(device)

optim = torch.optim.Adam(model.parameters(), lr = lr)

n, p = 0, 0
while n<=iterations:
  hprev = torch.zeros(hidden_size, device=device) # reset RNN memory

  # Why apply padding here
  inputs = add_padding(int(max_race_size),training_inputs[p])
  outputs = add_padding(int(max_race_size),training_outputs[p])

  # Extract next subsequence of characters
  inputs = torch.tensor(training_inputs[p],dtype=torch.long, device=device)
  targets = torch.tensor(training_outputs[p],dtype=torch.float32, device=device)

  # Compute loss for current subsequence
  loss, hprev = calculate_loss(model,inputs,targets,hprev,vocab_size)
  optim.zero_grad()
  loss.backward()
  optim.step()

  if n % 500 == 0:
    print('[{:}] Loss: {:.2f}'.format(n,loss.item()))
    check_accuracy()

  p += 1 # move data pointer
  n += 1 # iteration counter

  if p == int(max_race_size) - 1:
    p = 0

[0] Loss: 0.56
Total Accuracy: 86.3056851615204
Total No Pit Accuracy: 97.59737403133566
Total Pit Accuracy (+/- 3 LAPS): 17.447535383113713
Total Pit Predictions: 4098
[500] Loss: 0.12
Total Accuracy: 95.79185116942554
Total No Pit Accuracy: 97.45190122708681
Total Pit Accuracy (+/- 3 LAPS): 21.8944099378882
Total Pit Predictions: 644
[1000] Loss: 0.02
Total Accuracy: 95.79185116942554
Total No Pit Accuracy: 97.45190122708681
Total Pit Accuracy (+/- 3 LAPS): 21.8944099378882
Total Pit Predictions: 644
[1500] Loss: 0.15
Total Accuracy: 94.42182531427382
Total No Pit Accuracy: 97.47123607949302
Total Pit Accuracy (+/- 3 LAPS): 19.860017497812773
Total Pit Predictions: 1143
[2000] Loss: 0.19
Total Accuracy: 95.85723201283842
Total No Pit Accuracy: 97.34179127020087
Total Pit Accuracy (+/- 3 LAPS): 19.485294117647058
Total Pit Predictions: 544
[2500] Loss: 0.07
Total Accuracy: 95.85723201283842
Total No Pit Accuracy: 97.34179127020087
Total Pit Accuracy (+/- 3 LAPS): 19.485294117647058
To

In [101]:
def check_accuracy():
  sigmoid = nn.Sigmoid()
  total, matches = 0, 0
  no_pit_prediction_total, correct_no_pit_prediction = 0,0
  pit_prediction_total, correct_pit_prediction = 0,0

  for t in range(len(testing_inputs) - 1):
      inputs = add_padding(int(max_race_size),testing_inputs[t])
      outputs = add_padding(int(max_race_size),testing_outputs[t])

      for x in range(len(inputs) - 1):
        hprev = torch.zeros(hidden_size, device=device)
        hprev, y = model(torch.tensor([inputs[x]], device=device, dtype=torch.long), hprev)
        y = sigmoid(y.squeeze(0))

        if (y.item() > 0.5) == (outputs[x] == 1.0):
          matches += 1

        if (y.item() > 0.5):
          pit_prediction_total += 1
          if 1 in outputs[max(0, x - 2): min(int(max_race_size), x+4)]:
              correct_pit_prediction += 1
        else:
          no_pit_prediction_total += 1
          if y.floor().item() == outputs[x]:
              correct_no_pit_prediction += 1

        total += 1

  if total != 0:
    print(f"Total Accuracy: {matches / total * 100}")
  else:
    print("Total Accuracy: 0.0")

  if no_pit_prediction_total != 0:
    print(f"Total No Pit Accuracy: {correct_no_pit_prediction / no_pit_prediction_total * 100}")
  else:
    print(f"Total No Pit Accuracy: 0.0")

  if pit_prediction_total != 0:
    print(f"Total Pit Accuracy (+/- 3 LAPS): {correct_pit_prediction / pit_prediction_total * 100}")
  else:
    print(f"Total Pit Accuracy (+/- 3 LAPS): 0.0")

  print(f"Total Pit Predictions: {pit_prediction_total}")

In [102]:
check_accuracy()

Total Accuracy: 95.91666914321377
Total No Pit Accuracy: 97.31482878262182
Total Pit Accuracy (+/- 3 LAPS): 17.261904761904763
Total Pit Predictions: 504


## Modify code to help with overfitting to the majority class

In [103]:
# TRAINING
training_inputs = []
training_outputs = []

RaceCalender22 = pd.read_csv(BASE_PATH + r"2022/eventCalender2022.csv")
for _, row in RaceCalender22.iterrows():
    if row['EventFormat'] != 'testing':
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2022/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            if max(TempRaceLoadDriver['TyreLife']) > max_tyre_life:
                max_tyre_life = max(TempRaceLoadDriver['TyreLife'])

            stint_change_array = [0 if stint_array[i] == stint_array[i + 1] else 1
                                  for i in range(len(stint_array) - 1)] + [0]
            stint_change_array = np.array(stint_change_array)
            mandatory_change_made = [1 if stint_array[i] == 1 else 0
                                     for i in range(len(stint_array))]
            mandatory_change_made = np.array(mandatory_change_made)

            combined_array = np.stack([stint_change_array, mandatory_change_made], axis=1)

            training_inputs.append(combined_array)
            training_outputs.append(stint_change_array)

In [104]:
# TESTING
testing_inputs = []
testing_outputs = []

RaceCalender23 = pd.read_csv(BASE_PATH + r"2023/eventCalender2023.csv")
for _, row in RaceCalender23.iterrows():
    if row['EventFormat'] != 'testing':
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2023/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            if max(TempRaceLoadDriver['TyreLife']) > max_tyre_life:
                max_tyre_life = max(TempRaceLoadDriver['TyreLife'])

            stint_change_array = [0 if stint_array[i] == stint_array[i + 1]else 1
                                  for i in range(len(stint_array) - 1)] + [0]
            stint_change_array = np.array(stint_change_array)
            mandatory_change_made = [1 if stint_array[i] == 1 else 0
                                     for i in range(len(stint_array))]
            mandatory_change_made = np.array(mandatory_change_made)

            combined_array = np.stack([stint_change_array, mandatory_change_made], axis=1)

            testing_inputs.append(combined_array)
            testing_outputs.append(stint_change_array)

In [110]:
def calculate_loss_v2(model,inputs,targets,hprev,vocab_size):
  loss_func = nn.BCEWithLogitsLoss()
  seq_length = len(inputs)
  outputs = []
  for t in range(seq_length):
    # For each character in the input sequence, pass through RNN with previous hidden state
    hprev,y = model(inputs[t], hprev)
    # Gradually build up matrix of output logits of size seq_length * vocab_size - we want it at every time step do not want this.
    # Compute cross entropy loss for seq_length actual targets against estimated distributions
    # RuntimeError: result type Float can't be cast to the desired output type Long
    outputs.append(y)
    print(y)

  outputs = torch.stack(outputs).squeeze(1).squeeze(1)
  loss = loss_func(outputs ,targets)

  # For truncated backprop, the next subsequence will use the final hidden state
  # but will not backprop through it so we need to detach
  hprev = hprev.detach()

  return loss, hprev

In [111]:
# PADDING
def add_padding(max_race_size, sequence):
    if len(sequence) >= max_race_size:
        return sequence[:max_race_size]
    else:
        padding_shape = (max_race_size - sequence.shape[0],) + sequence.shape[1:]
        padding = np.zeros(padding_shape)
        return np.concatenate([sequence,padding])

In [112]:
class RNNv1c(nn.Module):
    def __init__(self,vocab_size,hidden_size):
        super(RNNv1c, self).__init__()
        self.embedding =  nn.Embedding(vocab_size, hidden_size)
        self.linear_h = nn.Linear(hidden_size,hidden_size)
        self.linear_y = nn.Linear(hidden_size,1)
        self.tanh = nn.Tanh()

    def forward(self,x,hprev):
        h = self.tanh(self.embedding(x) + self.linear_h(hprev))
        y = self.linear_y(h)
        return h,y

In [113]:
model = RNNv1c(vocab_size,hidden_size)
model.to(device)

optim = torch.optim.Adam(model.parameters(), lr = lr)

n, p = 0, 0
while n<=iterations:
  hprev = torch.zeros(hidden_size, device=device) # reset RNN memory

  # Why apply padding here
  inputs = add_padding(int(max_race_size),training_inputs[p])
  outputs = add_padding(int(max_race_size),training_outputs[p])

  # Extract next subsequence of characters
  inputs = torch.tensor(training_inputs[p],dtype=torch.long, device=device)
  targets = torch.tensor(training_outputs[p],dtype=torch.float32, device=device)

  # Compute loss for current subsequence
  loss, hprev = calculate_loss_v2(model,inputs,targets,hprev,vocab_size)
  optim.zero_grad()
  loss.backward()
  optim.step()

  if n % 500 == 0:
    print('[{:}] Loss: {:.2f}'.format(n,loss.item()))
    check_accuracy()

  p += 1 # move data pointer
  n += 1 # iteration counter

  if p == int(max_race_size) - 1:
    p = 0

tensor([[ 0.1678],
        [-0.1818]], grad_fn=<AddmmBackward0>)
tensor([[ 0.0886],
        [-0.2326]], grad_fn=<AddmmBackward0>)
tensor([[ 0.1051],
        [-0.2172]], grad_fn=<AddmmBackward0>)
tensor([[ 0.0840],
        [-0.2154]], grad_fn=<AddmmBackward0>)
tensor([[ 0.0911],
        [-0.2116]], grad_fn=<AddmmBackward0>)
tensor([[ 0.0897],
        [-0.2128]], grad_fn=<AddmmBackward0>)
tensor([[ 0.0886],
        [-0.2131]], grad_fn=<AddmmBackward0>)
tensor([[ 0.0884],
        [-0.2130]], grad_fn=<AddmmBackward0>)
tensor([[ 0.0886],
        [-0.2130]], grad_fn=<AddmmBackward0>)
tensor([[ 0.0887],
        [-0.2130]], grad_fn=<AddmmBackward0>)
tensor([[ 0.0886],
        [-0.2130]], grad_fn=<AddmmBackward0>)
tensor([[ 0.0886],
        [-0.2130]], grad_fn=<AddmmBackward0>)
tensor([[ 0.0886],
        [-0.2130]], grad_fn=<AddmmBackward0>)
tensor([[-0.1869],
        [-0.2130]], grad_fn=<AddmmBackward0>)
tensor([[0.0393],
        [0.0520]], grad_fn=<AddmmBackward0>)
tensor([[0.0706],
        [

ValueError: Target size (torch.Size([54])) must be the same as input size (torch.Size([54, 2, 1]))